In [18]:
import os

In [19]:
import dotenv

dotenv.load_dotenv() #WARNING: This will load the .env file in the current directory

POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB_NAME = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")


In [20]:
get_psycopg3_connection()

'host=None port=None dbname=None user=None password=None'

In [9]:
source = PostgresDataSource()

In [14]:
source.get_timeseries(lat=37.7749, lon=-122.4194)

OperationalError: invalid integer value "None" for connection option "port"


In [ ]:
from __future__ import annotations

import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Optional
from abc import ABC, abstractmethod
import psycopg2
import os
import dotenv

dotenv.load_dotenv() #WARNING: This will load the .env file in the current directory

POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB_NAME = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
CONTAINER_NAME = os.getenv("CONTAINER_NAME")



def get_psycopg3_connection():
    connection_string = (
        f"host={POSTGRES_HOST} "
        f"port={POSTGRES_PORT} "
        f"dbname={POSTGRES_DB_NAME} "
        f"user={POSTGRES_USER} "
        f"password={POSTGRES_PASSWORD}"
    )
    return connection_string

def fake_timeseries_data() -> pd.DataFrame:
    """using numpy to generate Sine wave data for a year at hourly intervals"""
    time = pd.date_range(start="2022-01-01", end="2022-12-31", freq="h")
    data = np.sin(np.linspace(0, 2 * np.pi, len(time)))
    return pd.DataFrame({"time": time, "total_precipitation": data})


# Abstract Strategy
class DataSource(ABC):
    @abstractmethod
    def get_timeseries(self, lat: float, lon: float) -> pd.DataFrame:
        pass

# Concrete Strategy - CSV File Source
@dataclass
class CSVFileSource(DataSource):
    file_path: str

    def __post_init__(self):
        # self.data = pd.read_csv(self.file_path)
        self.data = fake_timeseries_data()

    def get_timeseries(self, lat: float, lon: float) -> pd.DataFrame:
        # df_filtered = self.data[(self.data["latitude"] == lat) & (self.data["longitude"] == lon)]
        df_filtered = self.data
        return df_filtered.sort_values(by="time")

# Concrete Strategy - Postgres Data Source
@dataclass
class PostgresDataSource(DataSource):
    connection_string: Optional[str] = None

    def __post_init__(self):
        self.connection_string = get_psycopg3_connection()

    def get_timeseries(self, lat: float, lon: float) -> pd.DataFrame:
        query = """
        WITH nearest_location AS (
            SELECT latitude, longitude, location_id
            FROM weather
            ORDER BY (latitude - %s)^2 + (longitude - %s)^2
            LIMIT 1
        )
        SELECT DISTINCT ON (w.time) w.time, w.total_precipitation, w.latitude, w.longitude, w.location_id
        FROM weather w
        JOIN nearest_location nl ON w.location_id = nl.location_id
        ORDER BY w.time;
        """
        
        with psycopg2.connect(self.connection_string) as conn:
            df = pd.read_sql(query, conn, params=(lat, lon))
        return df

# Concrete Strategy - netCDF File Source
@dataclass
class NetCDFFileSource(DataSource):
    file_path: str

    def __post_init__(self):
        self.data = fake_timeseries_data()

    def get_timeseries(self, lat: float, lon: float) -> pd.DataFrame:
        # Read netCDF file and extract data
        return pd.DataFrame()

# Context Class to Use Different Strategies
@dataclass
class TimeSeriesFetcher:
    data_source: DataSource

    def get_data(self, lat: float, lon: float) -> pd.DataFrame:
        return self.data_source.get_timeseries(lat, lon)

# Example Usage
if __name__ == "__main__":
    # CSV Example
    csv_source = CSVFileSource("data.csv")
    fetcher = TimeSeriesFetcher(csv_source)
    print(fetcher.get_data(40.7128, -74.0060))
    
    # Postgres Example
    pg_source = PostgresDataSource("dbname=test user=postgres password=secret host=localhost")
    fetcher = TimeSeriesFetcher(pg_source)
    print(fetcher.get_data(40.7128, -74.0060))



                    time  total_precipitation
0    2022-01-01 00:00:00         0.000000e+00
1    2022-01-01 01:00:00         7.192290e-04
2    2022-01-01 02:00:00         1.438458e-03
3    2022-01-01 03:00:00         2.157686e-03
4    2022-01-01 04:00:00         2.876912e-03
...                  ...                  ...
8732 2022-12-30 20:00:00        -2.876912e-03
8733 2022-12-30 21:00:00        -2.157686e-03
8734 2022-12-30 22:00:00        -1.438458e-03
8735 2022-12-30 23:00:00        -7.192290e-04
8736 2022-12-31 00:00:00        -2.449294e-16

[8737 rows x 2 columns]


/var/folders/yh/g08b7w3d411gn1p_cp_409ym0000gn/T/ipykernel_40210/3438745503.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=(lat, lon))
